In [2]:
%pip install langchain openai langchain-community langchain-openai html2text playwright beautifulsoup4 trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.1 MB/s eta 0:00:00


In [3]:
!playwright install

173.7 MiB [] 0% 0.0s173.7 MiB [] 0% 4.7s173.7 MiB [] 1% 3.0s173.7 MiB [] 1% 2.6s173.7 MiB [] 2% 2.3s173.7 MiB [] 3% 2.2s173.7 MiB [] 4% 2.1s173.7 MiB [] 5% 2.1s173.7 MiB [] 6% 2.2s173.7 MiB [] 7% 2.1s173.7 MiB [] 8% 2.0s173.7 MiB [] 9% 2.0s173.7 MiB [] 10% 2.0s173.7 MiB [] 11% 2.0s173.7 MiB [] 12% 1.9s173.7 MiB [] 13% 1.9s173.7 MiB [] 14% 1.8s173.7 MiB [] 15% 1.8s173.7 MiB [] 16% 1.8s173.7 MiB [] 17% 1.7s173.7 MiB [] 18% 1.7s173.7 MiB [] 19% 1.6s173.7 MiB [] 19% 1.7s173.7 MiB [] 20% 1.7s173.7 MiB [] 21% 1.6s173.7 MiB [] 22% 1.6s173.7 MiB [] 23% 1.5s173.7 MiB [] 24% 1.5s173.7 MiB [] 25% 1.4s173.7 MiB [] 26% 1.5s173.7 MiB [] 28% 1.5s173.7 MiB [] 29% 1.4s173.7 MiB [] 30% 1.4s173.7 MiB [] 32% 1.3s173.7 MiB [] 33% 1.3s173.7 MiB [] 34% 1.2s173.7 MiB [] 35% 1.2s173.7 MiB [] 37% 1.2s173.7 MiB [] 38% 1.1s173.7 MiB [] 39% 1.1s173.7 MiB [] 40% 1.1s173.7 MiB [] 41% 1.1s173.7 MiB [] 42% 1.1s173.7 MiB [] 43% 1.0s173.7 MiB [] 45% 1.0s173.7 MiB [] 46% 0.9s173.7 MiB [] 48% 0.9s173.7 MiB [] 49% 0.9s173.

In [4]:
!pip install nest-asyncio
import nest_asyncio

nest_asyncio.apply()

In [6]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [21]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import Html2TextTransformer
import trafilatura
from langchain_core.documents import Document

# urls = ["https://docs.nestjs.com/custom-decorators"]
urls = ['https://n.news.naver.com/article/015/0005181264?sid=101']
loader = AsyncChromiumLoader(urls, user_agent="MyCrawler/1.0")
html_docs = loader.load()  # 렌더링된 HTML

# print(html_docs[0].page_content)

def extract_and_documentation(document: Document) -> Document:
  content = trafilatura.extract(document.page_content)
  return Document(page_content=content, metadata=document.metadata)

texts = [extract_and_documentation(d) for d in html_docs]

# 뉴스기사의 본문을 Chunk 단위로 쪼갬
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,     # 쪼개는 글자수
    chunk_overlap=300,   # 오버랩 글자수
    length_function=len,
    is_separator_regex=False,
)

# 웹사이트 내용 크롤링 후 Chunk 단위로 분할
docs = text_splitter.split_documents(texts)
print(docs)

# 각 Chunk 단위의 템플릿
template = '''다음의 내용을 한글로 요약해줘:

{text}
'''

# 전체 문서(혹은 전체 Chunk)에 대한 지시(instruct) 정의
combine_template = '''{text}

요약의 결과는 다음의 형식으로 작성해줘:
제목: 웹페이지 제목
주요내용: 한 줄로 요약된 내용
내용: 주요내용을 불렛포인트 형식으로 작성
'''

# 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=['text'])
combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])
print(prompt)
print(combine_prompt)

# LLM 객체 생성
llm = ChatOpenAI(temperature=0,
                 model_name='gpt-5-mini')

# 요약을 도와주는 load_summarize_chain
chain = load_summarize_chain(llm,
                             map_prompt=prompt,
                             combine_prompt=combine_prompt,
                             chain_type='map_reduce',
                             verbose=False)

# 실행결과
output_text = chain.invoke(docs).get('output_text')
print(output_text)

[Document(metadata={'source': 'https://n.news.naver.com/article/015/0005181264?sid=101'}, page_content="“삼성 인공지능(AI) 모델인 가우스를 자체적으로 고도화시키는 동시에 제품 기능이나 서비스에 따라 파트너사의 AI 모델을 적극적으로 활용하는 ‘하이브리드 AI’ 전략을 구사할 것입니다. ”\n노태문 삼성전자 디바이스경험(DX)부문장 직무대행(사장)은 유럽 최대 가전 전시회 'IFA2025' 개막 전날인 지난 4일(현지시간) 독일 베를린에서 연 기자간담회에서 삼성의 AI 전략에 대해 이같이 말했다.\n지난 4월 DX부문장 취임 후 처음으로 국내 언론과 만난 이날 자리에서 노 사장은 삼성 AI가 나아가야할 방향을 설명하는데 상당시간을 할애했다. 삼성전자는 자체 AI인 가우스와 함께 구글 제미나이 등 파트너사의 AI모델을 유연하게 활용하고 있다. 노 사장은 “AI의 발전과 전환기에 가장 중점을 두고 있는 부분은 DX부문 전체 사업 내 제품, 기능, 서비스에 빠르게 최신 AI를 적용해 고도화하는 것”이라고 말했다.\n하이브리드AI 전략에 대해선 “두 가지 의미가 있는데, 삼성전자가 보유한 기기에서 온디바이스 AI를 바탕으로 고객이 원하는 시점에 클라우드 AI를 적재적소에 사용하는 방식이 한 축”이라며 “또 다른 축은 가우스에 국한하지 않고, 파트너사의 AI 모델을 적극적으로 활용해 제품, 서비스에 따라 최적의 조합으로 제공하는 것”이라고 설명했다. 하드웨어와 소프트웨어를 같이 균형있게 결합해 서로 시너지를 내야 AI에 대한 최적의 경험과 서비스를 제공할 수 있다는 게 그의 생각이다.\n노 사장은 업무 측면에선 오는 2030년까지 AI를 활용해 'AI 선도 컴퍼니‘(AI Driven Company)로 거듭나겠다는 계획을 밝혔다. 그는 “AI를 중심으로 비즈니스 근본부터 혁신해 새로운 도약의 발판을 마련하겠다”며 “AI를 가장 잘 활용하는 회사, AI로 일하고 성장해나가는 회사로 빠르게 